## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


**Ответ**: 

- Чем они отличаются и чем они похожи?

User-User - даем рекомендации товаров пользователю похожие на товары похожих на соседних пользователей. Item-Item - даем рекомендации каким пользователям рекомендовать товар, используя соседние товары похожие на наш. Они похожи способом получения рекомендаций - из матрицы user - item мы создаем матрицу user-user (где на пересечении будут совпавшие товары) или item-item (на пересечении будут совпавшие пользователи).  Различаются направленностью действий - в случае user-user мы находим похожих пользователей, а в item-item - товары

- Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?

Можно, только нужно транспорнировать матрицу, чтобы по строкам получись юзеры, а по столбцам - товары.

- В чем принципиальные отличия item-item рекомендаций от ALS?

В item-item мы используем KNN, а в ALS матричное разложение и матрицы ембедингов юзеров и товаров, с дополнительными латентными факторами. В ALS так же используется факт, если даже пользователь не имел взаимодействия с товаром, в item-item используются только факты взаимодействия. В случае с ALS мы используем разложение матрицы на матрицу латентных факторов юзеров и матрицу латентных факторов товаров, а не используем ее всю. Это позволяет выиграть в стабильности и скорости получения рекомендаций


2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 

- количество просмотров страницы
- длительность пребывания на странице товара
- частота покупок
- популярные товары по дням недели или по рабочим/выходным
- категории товара
- переход/не переход по ссылке
- длительность просмотра видео
- добавление товара в корзину

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

- ALS не учитывает фичи юзеров и товаров, к примеру пол, цену, возраст и т.д.
- Не учитывается сезонность товаров и нет предпраздничных рекомендаций
- Для разложения UserItem матрицы на матрицы эмбедингов юзеров и товаров необходимо чтобы каждый пользователь сделал хотя бы одну покупку и чтобы каждый товар купили хотя бы один раз
- ALS не учитывает нелинейные взаимодействия, так как при расчете функции потерь эмбединги пользователей и товаров перемножаются скалярно


4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

**Ответ**:

1. Вводится зависимость релевантности от вхождения или не вхождения слов в запросах с более чем одного слова. Релевантность всего поискового запроса равна сумме релевантностей всех слов. Таким образом, отсутствие слова или другими словами image (его частота) равна 0 дает релевантность 0.

2. Преимущество при поиске в запросах с более чем 2-ух слов, одно из которых менее употребительно (более узкоспециализированное) будет отдаваться документам которые содержат это узкоспециализированное слово.

3. отсутствие учета внешних факторов (динамические/циклические составляющие временного ряда, смена тренда и т.д)

4. Для разложения UserItem матрицы на матрицы эмбедингов юзеров и товаров необходимо чтобы каждый пользователь сделал хотябы одну покупку и чтобы каждый товар купили хотябы один раз

5. ALS не учитывает нелинейные взаимодействия, так как при расчете функции потерь эмбединги пользователей и товаров перемножаются скалярно

Модификация:

При экстремальных значениях коэффициента ${\displaystyle b}$ в функции BM25 получаются функции ранжирования, известные под названиями BM11 (при ${\displaystyle b=1}$ и BM15 (при ${\displaystyle b=0}).$

источник - https://habr.com/ru/post/162937/

## Практическая часть


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

# Для работы с матрицами
from scipy.sparse import csr_matrix

import implicit

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('../../data/rec_sys/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../../data/rec_sys/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

Количество департаментов 44. Не так много. Будем арскрашивать по департаментам.

In [5]:
item_features.department.nunique()

44

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Для последовательности обучаем на 5000 товарах, но без проблем можно обучить и на всех 90 тыс.

In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
# Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # кол-во покупок. Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Максимальное количество покупок

In [9]:
user_item_matrix.max().max()

2926.0

In [10]:
user_item_matrix.shape

(2499, 5001)

In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
#?AlternatingLeastSquares

In [13]:
%%time

model = AlternatingLeastSquares(factors=100, # Нужно подбирать. Качество очень зависит.
                                # можно подбирать по кроссвалидации
                                regularization=0.001,
                                iterations=15, # кол-во итераций
                                calculate_training_loss=True, # отображать метрику loss в выводе
                                num_threads=4) # кол-во потоков распаралеливания

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=[itemid_to_id[999999]], 
                        # filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 4.7 s


Для повышения качества нужно либо менять веса, лио тюнить гиперпараметры.

Посмотрим как выглядят рекомендации. В большинстве случаев от 0 до 1, но могут и выходить за эти интервалы.

In [14]:
recs

[(2148, 0.8123470230059925),
 (2927, 0.7649155640241188),
 (2307, 0.7091293386948905),
 (4346, 0.6597187713947759),
 (2757, 0.65216827139097)]

In [15]:
[id_to_itemid[rec[0]] for rec in recs]

[981760, 1044078, 995242, 6534178, 1029743]

Функция конвертации прогноза модели в нужные item id

In [16]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    #filter_items=None, 
                                    recalculate_user=True)]
    return res

In [17]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


Запускаем ALS на всех юзерах и измеряем пресижн

In [18]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 49.1 s


0.18119490695396423

In [19]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.18119490695396423

С ходу без подборов гиперпараметров пресижн = 18% - выше, чем в моделях прошлого вебинара.

In [20]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[986912, 1033142, 1105488, 5569374, 979707]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 5568378, 910032, 951590]"


BM25 взвешивание

In [21]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [23]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()


Wall time: 49.5 s


0.22350636630753878

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


Вариант весов 1: nunique

In [24]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='nunique', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [26]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_nunique'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_nunique'], row['actual']), axis=1).mean()


Wall time: 53 s


0.212634671890301

Вариант весов 2: mean

In [27]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [29]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_mean'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_mean'], row['actual']), axis=1).mean()


Wall time: 39.7 s


0.21038197845249468

Вариант весов 3: quantity

In [30]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [32]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_quantity'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_quantity'], row['actual']), axis=1).mean()


Wall time: 56.2 s


0.18364348677766623

Вариант весов 3: size

In [33]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='size', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [35]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_size'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_size'], row['actual']), axis=1).mean()


Wall time: 51.8 s


0.22164544564152494

In [36]:
result

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[986912, 1033142, 1105488, 5569374, 979707]","[1082185, 995242, 1051211, 934369, 856942]","[1082185, 1135694, 8090541, 1061203, 872137]","[1082185, 1051211, 986912, 1056509, 847344]","[1082185, 6534178, 946276, 995242, 986912]","[1100972, 1082185, 995242, 1033142, 934369]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 5568378, 910032, 951590]","[1053690, 951590, 1092026, 1133018, 1106523]","[951590, 5569230, 910032, 9337581, 826249]","[6534178, 1029743, 1082185, 995242, 1133018]","[6534178, 844165, 1029743, 883404, 1053690]","[1133018, 1092026, 883404, 951590, 1053690]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 930118, 866211, 878996, 1051516]","[1023720, 1082185, 895268, 1024306, 878996]","[1082185, 878996, 1024306, 1127831, 866211]","[1082185, 6534178, 1127831, 981760, 878996]","[1023720, 1082185, 1024306, 1127831, 878996]","[1023720, 1082185, 1024306, 1051516, 895268]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 938700, 1133018, 1126899]","[1082185, 6944571, 1126899, 1058997, 1122358]","[1082185, 1127831, 857390, 1097544, 879635]","[857390, 1052046, 1082185, 849843, 6534178]","[1082185, 6534178, 1029743, 995242, 1106523]","[1082185, 1058997, 6944571, 1126899, 828867]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[844179, 938700, 1004906, 1053690, 951526]","[995242, 840361, 1068719, 965766, 1053690]","[1082185, 840361, 1068719, 883404, 1053763]","[1082185, 995242, 6534178, 1133018, 1106523]","[995242, 1082185, 6534178, 1029743, 1106523]","[1004906, 883404, 840361, 1068719, 995242]"
...,...,...,...,...,...,...,...,...
2037,2496,[6534178],"[914190, 1071939, 1013321, 938700, 871756]","[914190, 1133018, 904360, 916122, 1070702]","[844179, 1070702, 916122, 1000753, 1082185]","[6534178, 1082185, 995242, 916122, 981760]","[6534178, 1082185, 995242, 916122, 981760]","[914190, 840361, 916122, 6534178, 847982]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[5569845, 951590, 5569230, 5585510, 962568]","[957951, 951590, 5569230, 995785, 981760]","[981760, 5569230, 826249, 1082185, 995785]","[5569230, 5569471, 1082185, 6534178, 1029743]","[5569471, 5569230, 1022003, 1007195, 6534178]","[5569230, 5569845, 995785, 957951, 5569471]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1053690, 1070820, 1082185, 9526410, 9527290]","[995242, 862349, 840361, 1082185, 861272]","[1036432, 6514160, 840361, 1082185, 895070]","[6533889, 1055853, 202291, 927783, 972569]","[6533889, 202291, 1055853, 1081189, 878234]","[995242, 840361, 862349, 861272, 6534178]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[6534178, 1004906, 907631, 883404, 938700]","[5569845, 883404, 826249, 1098066, 6534178]","[826249, 883404, 1098066, 5569230, 1082185]","[1082185, 6534178, 6533889, 995242, 1029743]","[6533889, 6534178, 1082185, 995242, 1029743]","[826249, 883404, 1098066, 930917, 5569845]"


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [38]:
np.linspace(0.00001,1,11)

array([1.00000e-05, 1.00009e-01, 2.00008e-01, 3.00007e-01, 4.00006e-01,
       5.00005e-01, 6.00004e-01, 7.00003e-01, 8.00002e-01, 9.00001e-01,
       1.00000e+00])

In [ ]:
range(50, 550, 50)

In [43]:
for factors in tqdm(np.linspace(50,150,6)):
    for reg in np.linspace(0.0001,0.01,6):

        # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
        data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

        # заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
        user_item_matrix = pd.pivot_table(data_train, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробоват ьдругие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

        # переведем в формат saprse matrix
        sparse_user_item = csr_matrix(user_item_matrix).tocsr()

        user_item_matrix.head(3)

        user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

        

        model = AlternatingLeastSquares(factors=int(factors), 
                                        regularization=reg,
                                        iterations=15, 
                                        calculate_training_loss=True, 
                                        num_threads=8) # K - кол-во билжайших соседей

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                  show_progress=False)

        result[f'als_bm25_factors={factors}_reg={reg}'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        result.apply(lambda row: precision_at_k(row[f'als_bm25_factors_{factors}_reg{reg}'], row['actual']), axis=1).mean()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [44]:
result

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size,als_bm25_factors_50_reg1e-05,als_bm25_factors_50_reg0.100009,...,als_bm25_factors_130.0_reg0.00406,als_bm25_factors_130.0_reg0.00604,als_bm25_factors_130.0_reg0.00802,als_bm25_factors_130.0_reg0.01,als_bm25_factors_150.0_reg0.0001,als_bm25_factors_150.0_reg0.00208,als_bm25_factors_150.0_reg0.00406,als_bm25_factors_150.0_reg0.00604,als_bm25_factors_150.0_reg0.00802,als_bm25_factors_150.0_reg0.01
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[986912, 1033142, 1105488, 5569374, 979707]","[1082185, 995242, 1051211, 934369, 856942]","[1082185, 1135694, 8090541, 1061203, 872137]","[1082185, 1051211, 986912, 1056509, 847344]","[1082185, 6534178, 946276, 995242, 986912]","[1100972, 1082185, 995242, 1033142, 934369]","[1082185, 995242, 885290, 934369, 1100972]","[1082185, 995242, 1100972, 934369, 1024306]",...,"[1082185, 995242, 965766, 1051211, 9527290]","[1082185, 1051211, 995242, 965766, 1033142]","[995242, 1082185, 965766, 934369, 1033142]","[995242, 1082185, 1051211, 934369, 1033142]","[1082185, 995242, 1025641, 9527290, 1033142]","[995242, 1082185, 1051211, 965766, 856942]","[1082185, 965766, 995242, 1033142, 934369]","[995242, 1082185, 965766, 1004390, 1033142]","[1082185, 995242, 965766, 1033142, 856942]","[995242, 965766, 1082185, 934369, 856942]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 5568378, 910032, 951590]","[1053690, 951590, 1092026, 1133018, 1106523]","[951590, 5569230, 910032, 9337581, 826249]","[6534178, 1029743, 1082185, 995242, 1133018]","[6534178, 844165, 1029743, 883404, 1053690]","[1133018, 1092026, 883404, 951590, 1053690]","[951590, 1053690, 5569230, 844165, 962229]","[951590, 1053690, 910032, 5569230, 844165]",...,"[1133018, 1053690, 1106523, 1092026, 951590]","[1053690, 1133018, 951590, 1092026, 883404]","[1133018, 1106523, 1053690, 1092026, 951590]","[1133018, 1106523, 951590, 1092026, 1053690]","[1133018, 1053690, 1106523, 951590, 1092026]","[1133018, 1053690, 1106523, 1092026, 951590]","[1133018, 1106523, 1053690, 1092026, 951590]","[1133018, 951590, 1106523, 1053690, 1092026]","[1133018, 1092026, 1106523, 1053690, 951590]","[1133018, 1092026, 910032, 1106523, 1053690]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 930118, 866211, 878996, 1051516]","[1023720, 1082185, 895268, 1024306, 878996]","[1082185, 878996, 1024306, 1127831, 866211]","[1082185, 6534178, 1127831, 981760, 878996]","[1023720, 1082185, 1024306, 1127831, 878996]","[1023720, 1082185, 1024306, 1051516, 895268]","[1082185, 1024306, 857006, 834484, 911878]","[1082185, 1024306, 878996, 834484, 1051516]",...,"[1082185, 1023720, 878996, 1024306, 895268]","[1082185, 1023720, 1051516, 1024306, 878996]","[1082185, 1023720, 1024306, 878996, 1051516]","[1082185, 878996, 1024306, 1051516, 866211]","[878996, 919681, 1082185, 1024306, 1007195]","[878996, 1023720, 1082185, 1051516, 1024306]","[878996, 1082185, 1051516, 1024306, 1023720]","[1082185, 878996, 1007195, 1024306, 1023720]","[1082185, 866211, 895268, 1024306, 878996]","[1098248, 1082185, 878996, 1007195, 1024306]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 938700, 1133018, 1126899]","[1082185, 6944571, 1126899, 1058997, 1122358]","[1082185, 1127831, 857390, 1097544, 879635]","[857390, 1052046, 1082185, 849843, 6534178]","[1082185, 6534178, 1029743, 995242, 1106523]","[1082185, 1058997, 6944571, 1126899, 828867]","[1082185, 6944571, 849843, 1058997, 1130111]","[849843, 1082185, 6944571, 889731, 1058997]",...,"[1082185, 6944571, 828867, 1126899, 1058997]","[1082185, 6944571, 1058997, 1126899, 1122358]","[1126899, 1082185, 6944571, 1058997, 828867]","[1082185, 6944571, 1126899, 1058997, 828867]","[1082185, 849843, 6944571, 1126899, 1058997]","[1082185, 6944571, 1126899, 1058997, 826249]","[1126899, 1082185, 1058997, 6944571, 828867]","[1082185, 6944571, 828867, 1126899, 1122358]","[1082185, 6944571, 1058997, 

In [51]:
df_res = pd.DataFrame(columns=['parameters', 'presision_at_k'])
df_res
df_res

,parameters,presision_at_k


In [1]:
for column in result.columns[2:]:
    print(column, round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3))

NameError: name 'result' is not defined

In [ ]:
df_res = pd.DataFrame(columns=['parameters', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    #print(column, a)
    #df = pd.DataFrame([column, a])
    df_res.loc[i] = [column, a]
    i += 1
df_res

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!